In [ ]:
import copy

In [1]:
test_input = """Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1"""

In [2]:
import requests

URL = "https://adventofcode.com/{year}/day/{day}/input"
url = URL.format(year='2022', day=11)
cookies = {"session":"53616c7465645f5fd345d8c12076a357c8d31b2c0c4ad1c152eefbd5c1cf09e0c8d6d7f657fc558887d443d2d07fbc90728ada9b6a91afcecb9926ffb4981a93"}
req = requests.get(url, cookies=cookies)

if req.status_code != 200:
    print("Error. Got status:", req.status_code)
    print(req.text)
    exit(3)
else:
    my_input = req.text

In [3]:
class S4 : # stuff_slinging simian shenanigans

    def __init__(self, this_input) :
        
        monkeys = []

        for t in this_input.split("\n\n") :
            l = t.split("\n")
            num = int(l[0][7:-1])
            items = eval("["+l[1][18:]+"]")
            op = l[2][19:]
            test = "new//"+l[3][21:]+" == new/"+l[3][21:]
            m_true = int(l[4][29:])
            m_false = int(l[5][30:])
            monkeys.append({
                'num' : num,
                'items' : items,
                'op' : op ,
                'test' : test ,
                'm_true' : m_true,
                'm_false' : m_false,
                "inspected" : 0
            })
        
        self.monkeys = monkeys
        
    def __str__(self) :
        s = ""
        for m in self.monkeys :
            s += "Monkey "+str(m["num"]) + " : " + str(m["items"]) + " - inspected " + str(m["inspected"]) + " items\n"
        return s
    
    def pass_one_item(self,mNum) : 
        m = self.monkeys[mNum]
        if len(m["items"]) == 0 :
            return False
        old = m["items"].pop(0)
        new = eval(m["op"])
        new = new//3
        if eval(m["test"]) :
            next_m = m['m_true']
        else : 
            next_m = m['m_false']
        m['inspected'] += 1
        self.monkeys[next_m]['items'].append(new)
#         print(mNum,old,new,next_m)
        return True
    
    def one_turn(self,mNum) :
        while(self.pass_one_item(mNum)) :
            continue
        return True
    
    def one_round(self):
        for i in range(len(self.monkeys)):
            self.one_turn(i)
        return True
    
    def get_monkey_power(self,Nrounds):
        for i in range(Nrounds):
            self.one_round()
        return [m['inspected'] for m in self.monkeys]
    
    def get_result_part1(self):
        p = self.get_monkey_power(20)
        p.sort(reverse=True)
        return p[0]*p[1]

In [ ]:
m = S4(test_input)

In [ ]:
print(m)

In [ ]:
m.get_result_part1()

In [ ]:
m = S4(my_input)

In [ ]:
print(m)

In [ ]:
m.get_result_part1()

In [28]:
class S4_part2 : # stuff_slinging simian shenanigans

    def __init__(self, this_input) :
        
        monkeys = []
        starting_worries = []
        item_num = 0
        divisions = []

        for t in this_input.split("\n\n") :
            l = t.split("\n")
            num = int(l[0][7:-1])
            items = eval("["+l[1][18:]+"]")
            starting_worries.extend(items)
            items = [item_num + i for i in range(len(items))]
            item_num += len(items)
            op = l[2][19:]
            div_test = int(l[3][21:])
            divisions.append(div_test)
            m_true = int(l[4][29:])
            m_false = int(l[5][30:])
            monkeys.append({
                'num' : num,
                'items' : items,
                'op' : op ,
                'div_test' : div_test ,
                'm_true' : m_true,
                'm_false' : m_false,
                "inspected" : 0
            })
        
        self.monkeys = monkeys
        self.divisions = divisions
        self.items =  [ [ w % i for i in divisions ] for w in starting_worries ]
        
    def __str__(self) :
        s = ""
        for m in self.monkeys :
            s += "Monkey "+str(m["num"]) + " : " + str(m["items"]) + " - inspected " + str(m["inspected"]) + " items\n"
        s += "\nItems\n"
        for i in self.items :
            s += "item worry rests : "+str(i)+"\n"
        return s
    
    def pass_one_item(self,mNum) : 
        m = self.monkeys[mNum]
        if len(m["items"]) == 0 :
            return False
        item_id = m["items"].pop(0)
        
        item_rests = self.items[item_id]
        for i in range(len(self.divisions)) :
            old = item_rests[i]
            new = eval(m["op"])
            item_rests[i] = new % self.divisions[i]
        
        if item_rests[mNum] == 0 :
            next_m = m['m_true']
        else : 
            next_m = m['m_false']
        m['inspected'] += 1
        self.monkeys[next_m]['items'].append(item_id)
#         print(mNum,old,new,next_m)
        return True
    
    def one_turn(self,mNum) :
        while(self.pass_one_item(mNum)) :
            continue
        return True
    
    def one_round(self):
        for i in range(len(self.monkeys)):
            self.one_turn(i)
        return True
    
    def get_monkey_power(self,Nrounds):
        for i in range(Nrounds):
            self.one_round()
        return [m['inspected'] for m in self.monkeys]
    
    def get_result_part1(self):
        p = self.get_monkey_power(20)
        p.sort(reverse=True)
        return p[0]*p[1]
    
    def get_result_part2(self):
        p = self.get_monkey_power(10000)
        p.sort(reverse=True)
        return p[0]*p[1]

In [32]:
m = S4_part2(test_input)
print(m)
print("")
print(m.get_result_part2())
print("")
print(m)


Monkey 0 : [0, 1] - inspected 0 items
Monkey 1 : [2, 3, 4, 5] - inspected 0 items
Monkey 2 : [6, 7, 8] - inspected 0 items
Monkey 3 : [9] - inspected 0 items

Items
item worry rests : [10, 3, 1, 11]
item worry rests : [6, 3, 7, 13]
item worry rests : [8, 16, 2, 3]
item worry rests : [19, 8, 0, 14]
item worry rests : [6, 18, 10, 7]
item worry rests : [5, 17, 9, 6]
item worry rests : [10, 3, 1, 11]
item worry rests : [14, 3, 8, 9]
item worry rests : [5, 2, 6, 12]
item worry rests : [5, 17, 9, 6]


2713310158

Monkey 0 : [5, 9, 8, 3, 0] - inspected 52166 items
Monkey 1 : [1, 2, 6, 4, 7] - inspected 47830 items
Monkey 2 : [] - inspected 1938 items
Monkey 3 : [] - inspected 52013 items

Items
item worry rests : [13, 9, 0, 16]
item worry rests : [11, 3, 4, 13]
item worry rests : [14, 3, 11, 10]
item worry rests : [16, 9, 4, 16]
item worry rests : [19, 3, 7, 5]
item worry rests : [7, 9, 6, 5]
item worry rests : [2, 3, 10, 6]
item worry rests : [3, 3, 7, 7]
item worry rests : [4, 9, 7, 7]
item

In [33]:
m = S4_part2(my_input)
print(m)
print("")
print(m.get_result_part2())
print("")
print(m)


Monkey 0 : [0, 1, 2, 3, 4, 5] - inspected 0 items
Monkey 1 : [6, 7, 8, 9, 10] - inspected 0 items
Monkey 2 : [11, 12, 13] - inspected 0 items
Monkey 3 : [14, 15, 16, 17, 18, 19, 20, 21] - inspected 0 items
Monkey 4 : [22, 23, 24, 25] - inspected 0 items
Monkey 5 : [26, 27] - inspected 0 items
Monkey 6 : [28, 29, 30, 31, 32, 33, 34] - inspected 0 items
Monkey 7 : [35] - inspected 0 items

Items
item worry rests : [2, 3, 0, 8, 1, 0, 1, 14]
item worry rests : [3, 1, 0, 1, 0, 0, 10, 2]
item worry rests : [4, 2, 1, 2, 1, 1, 11, 3]
item worry rests : [3, 0, 11, 8, 0, 1, 12, 6]
item worry rests : [1, 2, 12, 7, 0, 1, 0, 13]
item worry rests : [4, 3, 3, 6, 1, 0, 9, 18]
item worry rests : [2, 0, 12, 0, 2, 1, 9, 1]
item worry rests : [4, 3, 3, 6, 1, 0, 9, 18]
item worry rests : [0, 0, 5, 4, 1, 0, 2, 13]
item worry rests : [3, 6, 5, 6, 2, 1, 15, 7]
item worry rests : [3, 4, 1, 9, 2, 1, 2, 15]
item worry rests : [3, 0, 7, 10, 2, 0, 13, 3]
item worry rests : [0, 1, 11, 6, 2, 0, 16, 12]
item worry re